In [1]:
import pandas as pd
import sys
sys.path.append("..")
from src import passwords as ps
from src import sp_analisis as sa

##  Ejercicio 1. Familiarizarse con la BBDD
![esquemaNorthwind](../data/esquemaNorthwind.png)

## Ejercicio 2. Primeras consultas

- ### ¿Cuántos empleados tenemos contratados en 'Global Importaciones'? Indica su id, nombre, apellido, ciudad y país.

In [2]:
query_employee = ''' SELECT  "employee_id", 
                              CONCAT("first_name", ' ', "last_name") AS "employee_name",
                              "city", 
                              "country"
                      FROM "employees";
                  '''
df_Employee= sa.pasos_querys(query_employee)
df_Employee

,employee_id,employee_name,city,country
0,1,Nancy Davolio,Seattle,USA
1,2,Andrew Fuller,Tacoma,USA
2,3,Janet Leverling,Kirkland,USA
3,4,Margaret Peacock,Redmond,USA
4,5,Steven Buchanan,London,UK
5,6,Michael Suyama,London,UK
6,7,Robert King,London,UK
7,8,Laura Callahan,Seattle,USA
8,9,Anne Dodsworth,London,UK


In [14]:
print(f'En Global Importaciones tenemos contratados a {df_Employee["employee_id"].count()} personas')

En Global Importaciones tenemos contratados a 9 personas


- ### ¿Qué productos tenemos? Indica el id del producto, id del proveedor, nombre del producto, precio por unidad, unidades en stock, unidades pedidas al proveedor y productos descontinuados

In [3]:
query_products = ''' SELECT  "product_id",
                              "supplier_id",
                              "category_id",
                              "product_name",
                              "unit_price",
                              "units_in_stock",
                              "units_on_order",
                              "discontinued"
                      FROM "products";
                  '''
df_Products= sa.pasos_querys(query_products)
df_Products.sample()

,product_id,supplier_id,category_id,product_name,unit_price,units_in_stock,units_on_order,discontinued
57,58,27,8,Escargots de Bourgogne,13.25,62,0,0


In [16]:
print(f'En Global Importaciones tenemos {df_Products["product_id"].count()} productos')

En Global Importaciones tenemos 77 productos


In [17]:
df_Products.to_csv('../data/products.csv', index=False)

- ### ¿Tenemos productos descontinuados?. Indica el nombre del producto, y cantidad que nos queda en stock. 

In [4]:
query_discontinued = ''' SELECT "product_name",
                              "units_in_stock",
                              "discontinued"
                          FROM "products"
                          WHERE "discontinued" = 1;
                      '''
df_Products_discontinued = sa.pasos_querys(query_discontinued)
df_Products_discontinued.sample()

,product_name,units_in_stock,discontinued
5,Guaraná Fantástica,20,1


In [5]:
print(f'Tenemos {df_Products_discontinued ["discontinued"].count()} productos descontinuados')

Tenemos 10 productos descontinuados


- ### ¿Qué proveedores tenemos? Indica el id de la compañía, nombre de la compañía, ciudad y país.

In [6]:
query_suppliers = ''' SELECT "supplier_id",
                              "company_name",
                              "city",
                              "country"
                        FROM "suppliers"                    
                  '''

df_Suppliers = sa.pasos_querys(query_suppliers)
df_Suppliers.sample()

,supplier_id,company_name,city,country
13,14,Formaggi Fortini s.r.l.,Ravenna,Italy


In [7]:
print(f'Tenemos un total de {df_Suppliers["supplier_id"].count()} proveedores')

Tenemos un total de 29 proveedores


In [22]:
df_Suppliers.to_csv('../data/suppliers.csv', index=False)

- ### ¿Qué pedidos hemos tenido? Indica el número de pedido, id del cliente,id del transportista, dia del pedido, día requerido de llegada y día de llegada real.

In [8]:
query_orders = ''' SELECT "order_id",
                          "customer_id",
                          "ship_via",
                          "order_date",
                          "required_date",
                          "shipped_date"
                    FROM "orders"
               '''
df_Orders = sa.pasos_querys(query_orders)
df_Orders.sample()

,order_id,customer_id,ship_via,order_date,required_date,shipped_date
135,10383,AROUT,3,1996-12-16,1997-01-13,1996-12-18


In [24]:
df_Orders.to_csv('../data/orders.csv', index=False)

- ### ¿Cuántos pedidos hemos tenido?

In [25]:
print(f'Hemos tenido {df_Orders["order_id"].count()} pedidos')

Hemos tenido 830 pedidos


- ### ¿Cuántos clientes tenemos? Indica el id del cliente, nombre de la compañía, ciudad y país.

In [9]:
query_customers = ''' SELECT "customer_id",
                              "company_name",
                              "city",
                              "country"
                      FROM "customers"
                  '''
df_Customers = sa.pasos_querys(query_customers)
df_Customers.sample()

,customer_id,company_name,city,country
87,WELLI,Wellington Importadora,Resende,Brazil


In [10]:
print(f'Tenemos un total de {df_Customers["customer_id"].count()} clientes')

Tenemos un total de 91 clientes


In [28]:
df_Customers.to_csv('../data/customers.csv', index=False)

- ### ¿Con qué empresas de transporte trabajamos? Indica su id del transportista y el nombre de la compañía.

In [11]:
query_shippers = ''' SELECT "shipper_id",
                            "company_name"
                    From "shippers"                  
                  '''
df_Shippers = sa.pasos_querys(query_shippers)
df_Shippers.sample()

,shipper_id,company_name
3,4,Alliance Shippers


In [30]:
print(f'Trabajamos con un total de {df_Shippers["shipper_id"].count()} empresas de transporte')

Trabajamos con un total de 6 empresas de transporte


In [31]:
df_Shippers.to_csv('../data/shippers.csv', index=False)

- ### ¿Cómo son las relaciones de reporte de resultados entre los empleados?

In [12]:
query_reports = ''' SELECT "employee_id",
                            CONCAT("first_name", ' ', "last_name") AS "employee_name",
                            "reports_to"
                      FROM "employees";    
                '''
df_Reports = sa.pasos_querys(query_reports)
df_Reports.sample()

,employee_id,employee_name,reports_to
3,4,Margaret Peacock,2.0


In [13]:
# Hago un self-join para obtener el nombre del supervisor
df_Reports = df_Reports.merge(df_Reports[['employee_id', 'employee_name']], 
                              left_on='reports_to', 
                              right_on='employee_id', 
                              how='left', 
                              suffixes=('', '_manager'))

# Renombro la columna con el nombre del supervisor
df_Reports.rename(columns={'employee_name_manager': 'manager_name'}, inplace=True)
df_Reports

,employee_id,employee_name,reports_to,employee_id_manager,manager_name
0,1,Nancy Davolio,2.0,2.0,Andrew Fuller
1,2,Andrew Fuller,NaN,NaN,NaN
2,3,Janet Leverling,2.0,2.0,Andrew Fuller
3,4,Margaret Peacock,2.0,2.0,Andrew Fuller
4,5,Steven Buchanan,2.0,2.0,Andrew Fuller
5,6,Michael Suyama,5.0,5.0,Steven Buchanan
6,7,Robert King,5.0,5.0,Steven Buchanan
7,8,Laura Callahan,2.0,2.0,Andrew Fuller
8,9,Anne Dodsworth,5.0,5.0,Steven Buchanan


In [14]:
# Contar cuántos empleados tiene cada supervisor
supervisores = df_Reports.groupby('manager_name')['employee_id'].count()

# Crear el mensaje para cada supervisor
for supervisor, num_empleados in supervisores.items():
    print(f'El supervisor {supervisor} tiene {num_empleados} empleados.')


El supervisor Andrew Fuller tiene 5 empleados.
El supervisor Steven Buchanan tiene 3 empleados.
